**Download and Import Packages**
- Pandas, Plotly, and Dash are the main packages we use for this project.

In [1]:
!pip install dash
!pip install dash_mantine_components

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash
import dash_mantine_components as dmc
from dash import dcc, html, Input, Output, callback

**Preliminary Cell**
- The preliminary cell allows users to change variable names correspoding to the columns in their raw data, so that users won't have to do so one by one in the main dash app.
- Mock transaction records are made specifically for this demo for NDA. The data follows uniform distribution as you might observe in some of the graphs below.

In [3]:
DS_PATH = '/content/mock_agreements.pq'
CUST_ID_COL = 'CFK_CIF_NBR'.lower()
START_DATE_COL = 'S_OPENING_DT'.lower()
CLOSE_DATE_COL = 'S_CLOSE_DATE'.lower()
OPENING_AMOUNT_COL = 'S_OPENING_AMT'.lower()
PRODUCT_TYPE = 'S_ACCT_TYPE'.lower()
DATE_DIFF = 'DATE_DIFF'.lower()

**Data Preprocessing and Sampling**
- Here, we transform time-related columns to datetime format.
- Also, we check for duplicate records in dataset.

In [4]:
df = pd.read_parquet(DS_PATH)

# Convert date columns to datetime format
df[START_DATE_COL] = pd.to_datetime(df[START_DATE_COL], errors='coerce')
df[CLOSE_DATE_COL] = pd.to_datetime(df[CLOSE_DATE_COL], errors='coerce')

# Convert amount column to float format
df[OPENING_AMOUNT_COL] = pd.to_numeric(df[OPENING_AMOUNT_COL], errors='coerce').astype(float)

# Check for duplicated rows
if len(df[df.duplicated()]) > 0:
    df = df.drop_duplicates()
    print('Duplicated Records Removed!')
else:
    print('No duplicated Records Found!')

# Random Sampling to 0.5M rows
df = df.sample(n=500000)

Duplicated Records Removed!


**Main App and Dashboards**
<br>The dashboard is composed of 4 tabs- each presenting the dataset in different perspective.
1. General Statistics: This tab provides descriptive statistics of the dataset, such as the number of unique client, maximum/minimum transaction amount, latest/earliest transaction date.

2. Event Count: This tab provides deeper insights on the numbers of transactions over selectable time period, such as how many transactions are made in each day, month, or year?

3. Event Value: This tab serves to understand what are the average transaction values over selectable time period, such as what is the average transaction value on the day Thursday?

4. Time Difference: This tab explores customer behaviors by observing the time difference of transactions made by each customer. This gives us some clues on the product attractiveness.

In [5]:
# Create the main Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Layout for Tab1: General Stats
app0 = dash.Dash(__name__)
app0.layout = html.Div(
    [
        html.H1('General Statistics', style={'textAlign': 'center'}),
        html.P("Number of unique clients: " + "{:,}".format(df[CUST_ID_COL].nunique())),
        html.P("Total number of events: " + "{:,}".format(df.shape[0])),
        html.P("Earliest event date: " + str(df[START_DATE_COL].min())),
        html.P("Latest event date: " + str(df[START_DATE_COL].max())),
        html.P("Average event value: " + "{:,}".format(round(df[OPENING_AMOUNT_COL].mean(),2))),
        html.P("Maximum event value recorded: " + "{:,}".format(round(df[OPENING_AMOUNT_COL].max(),2))),
        html.P("Minimum event value recorded: " + "{:,}".format(round(df[OPENING_AMOUNT_COL].min(),2))),
        html.P("Most frequent event value: " + "{:,}".format(round(df[OPENING_AMOUNT_COL].mode().iloc[0],2))),
        html.P("Least frequent event value: " + "{:,}".format(round(df[OPENING_AMOUNT_COL].value_counts().index[-1],2))),
        dcc.Dropdown(id='product-dropdown', options=[{'label': str(product), 'value': product} for product in sorted(df[PRODUCT_TYPE].unique())],
                     multi=True, placeholder='Select Product(s)'),
        dcc.Graph(id='overall-distribution')
    ]
)

@app.callback([Output('overall-distribution', 'figure')],
              [Input('product-dropdown', 'value')]
)

# Violin Chart showing the distribution of transactions by product
def overall_distr_fig(selected_product):
    overall_distr_fig = go.Figure(data=go.Violin(y=df[OPENING_AMOUNT_COL], box_visible=True,
                                                  meanline_visible=True, x0='Overall', name='Overall'))
    overall_distr_fig.update_layout(
            title='Overall Event Value Distribution by Product',
            title_x=0.5,
            xaxis_title='Product Type',
            yaxis_title='Event Value')

    if selected_product:
        for product in selected_product:
            overall_distr_fig.add_trace(go.Violin(y=df[df[PRODUCT_TYPE]==product][OPENING_AMOUNT_COL], box_visible=True,
                                                    meanline_visible=True, x0=str(product), name=str(product)))

    return [overall_distr_fig]

# Layout for Tab2: Event Count
app1 = dash.Dash(__name__)
app1.layout = html.Div(
    [
        html.H1('Event Count vs Time', style={'textAlign': 'center'}),
        html.Div(
                [dmc.DateRangePicker(
                    id="date-picker-range",
                    label="Date Range",
                    minDate=pd.Timestamp(df[START_DATE_COL].min()),
                    maxDate=pd.Timestamp(df[START_DATE_COL].max()),
                    value=[pd.Timestamp(df[START_DATE_COL].min()), pd.Timestamp(df[START_DATE_COL].max())],
                    style={"width": 350},
                ),
                dmc.Space(h=10)]
        ),
        html.P("Event Value Threshold Rangeslider", style={'textAlign': 'center'}),
        dcc.RangeSlider(
            min=df[OPENING_AMOUNT_COL].min(),
            max=df[OPENING_AMOUNT_COL].max(),
            step=(df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10,
            id='threshold-slider',
            value=[df[OPENING_AMOUNT_COL].min(), df[OPENING_AMOUNT_COL].max()],
            marks={i: str(i) for i in range(int(df[OPENING_AMOUNT_COL].min()), int(df[OPENING_AMOUNT_COL].max()), round((df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10))}
        ),
        html.Button('Month/Year/Day', id='day-button', n_clicks=0),
        html.P("How many events occurred each day/month/year? (With Event Value Threshold)"),
        dcc.Graph(id='event-count-chart'),
        html.P("How many unique clients opened an event each day/month/year"),
        dcc.Graph(id='unique-client-chart'),
        dcc.Dropdown(id='product-dropdown', options=[{'label': str(product), 'value': product} for product in sorted(df[PRODUCT_TYPE].unique())],
                    placeholder='Select Product(s)'),
        html.P("How many events did each client open? (Overall/Month/Year/Specific Time)"),
        html.P("Out of the number of times, how many is the first, second or third agreements?"),
        html.Button('Overall/Month/Year', id='day-button-client', n_clicks=0),
        dcc.Graph(id='event-count-client'),
        html.P("How many customers have multiple agreements with each month and year?"),
        html.Button('Month/Year', id='day-button-multiple-event', n_clicks=0),
        dcc.Graph(id='multiple-event-client'),
    ]
)

@app.callback([Output('event-count-chart', 'figure'),
               Output('unique-client-chart', 'figure')],
              [Input('date-picker-range', 'value'),
               Input('threshold-slider', 'value'),
               Input('day-button', 'n_clicks')]
)

def event_count_unique_customer(date_range, threshold_range, day_clicks):
    # Filter the dataframe based on the slider range and selected dates
    starting_date, ending_date = date_range
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    # Determine the grouping based on button clicks
    if day_clicks % 3 == 0:
        grouping = filtered_df[START_DATE_COL].dt.strftime('%Y-%m')
        x_title = 'Month'
        dtick_format = "M1"
        tick_format = "%b\n%Y"

    elif day_clicks % 3 == 1:
        grouping = filtered_df[START_DATE_COL].dt.year
        x_title = 'Year'
        dtick_format = "Y1"
        tick_format = "%b\n%Y"
    else:
        grouping = filtered_df[START_DATE_COL]
        x_title = 'Day'
        dtick_format = None
        tick_format = None

    # Create the event count chart
    event_count_fig = go.Figure()
    for product in sorted(filtered_df[PRODUCT_TYPE].unique()):
        event_counts = filtered_df[filtered_df[PRODUCT_TYPE]==product].groupby(grouping).size()
        x = event_counts.index
        y = event_counts.values
        name = str(product)
        event_count_fig.add_trace(go.Bar(x=x, y=y, name=name))

    event_count_fig.update_layout(
        xaxis_title=x_title,
        yaxis_title='Event Count',
        title='Event Count vs Time<br>Threshold: {}'.format(threshold_range),
        title_x=0.5,
        barmode='stack'
    )
    event_count_fig.update_xaxes(dtick=dtick_format, tickformat=tick_format)

    # Create the unique customers chart
    unique_customer_fig = go.Figure()
    for product in sorted(filtered_df[PRODUCT_TYPE].unique()):
        unique_customers = filtered_df[filtered_df[PRODUCT_TYPE]==product].groupby(grouping)[CUST_ID_COL].nunique()
        x = unique_customers.index
        y = unique_customers.values
        name = str(product)
        unique_customer_fig.add_trace(go.Bar(x=x, y=y, name=name))

    unique_customer_fig.update_layout(
        xaxis_title=x_title,
        yaxis_title='Unique Customer',
        title='Unique Customer vs Time<br>Threshold: {}'.format(threshold_range),
        title_x=0.5,
        barmode='stack'
    )
    unique_customer_fig.update_xaxes(dtick=dtick_format, tickformat=tick_format)

    return event_count_fig, unique_customer_fig

@app.callback(
    [Output('event-count-client', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('threshold-slider', 'value'),
     Input('product-dropdown', 'value'),
     Input('day-button-client', 'n_clicks')]
)

def event_count_client(date_range, threshold_range, selected_product, day_clicks_client):
    # Filter the dataframe based on the slider range and selected dates
    starting_date, ending_date = date_range
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Event count separated into first, second, third...times
    filtered_df_cust = filtered_df.sort_values(by=[CUST_ID_COL, START_DATE_COL]).copy()
    filtered_df_cust['rank'] = filtered_df_cust.groupby(CUST_ID_COL)[START_DATE_COL].rank(ascending=True, method='first')

    if day_clicks_client % 3 == 0:
        temp = filtered_df_cust.groupby(CUST_ID_COL).size().value_counts()
        event_count_client_fig = go.Figure(data=go.Bar(x=temp.index, y=temp.values))
        event_count_client_fig.update_layout(xaxis=dict(tickmode='array', tickvals=temp.index, ticktext=temp.index),
                                             xaxis_title='Number of Agreements', yaxis_title='Number of Customers',
                                             title='How many agreements did each customer open?',
                                             title_x=0.5)
    # Each month
    elif day_clicks_client % 3 == 1:
        event_count_client_fig = go.Figure()
        for group in sorted(filtered_df_cust['rank'].unique()):
            temp = filtered_df_cust[filtered_df_cust['rank']==group].groupby(filtered_df_cust[START_DATE_COL].dt.strftime('%Y-%m')).size()
            x = temp.index
            y = temp.values
            name = str(int(group)) + 'Agreement'
            event_count_client_fig.add_trace(go.Bar(x=x, y=y, name=name))

        event_count_client_fig.update_layout(
        title='Out of the Event Counts, how many is the first, second and third agreements?',
        yaxis_title='Event Count',
        title_x=0.5,
        barmode='stack')
        event_count_client_fig.update_xaxes(dtick='M1', tickformat="%b\n%Y")

    # Each year
    else:
        event_count_client_fig = go.Figure()
        for group in sorted(filtered_df_cust['rank'].unique()):
            temp = filtered_df_cust[filtered_df_cust['rank']==group].groupby(filtered_df_cust[START_DATE_COL].dt.year).size()
            x = temp.index
            y = temp.values
            name = str(int(group)) + 'Agreement'
            event_count_client_fig.add_trace(go.Bar(x=x, y=y, name=name))

        event_count_client_fig.update_layout(
        title='Out of the Event Counts, how many is the first, second and third agreements?',
        yaxis_title='Event Count',
        title_x=0.5,
        barmode='stack')
        event_count_client_fig.update_xaxes(dtick='Y1', tickformat="%b\n%Y")

    return [event_count_client_fig]

@app.callback(
    [Output('multiple-event-client', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('threshold-slider', 'value'),
     Input('product-dropdown', 'value'),
     Input('day-button-multiple-event', 'n_clicks')]
)

def multiple_event_client(date_range, threshold_range, selected_product, day_clicks_multiple_event):
    # Filter the dataframe based on the slider range and selected dates
    starting_date, ending_date = date_range
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Event count separated into first, second, third...times
    filtered_df_cust = filtered_df.sort_values(by=[CUST_ID_COL, START_DATE_COL]).copy()
    filtered_df_cust['rank'] = filtered_df_cust.groupby(CUST_ID_COL)[START_DATE_COL].rank(ascending=True, method='first')

    # How many customer have multiple agreements in each month and year
    if day_clicks_multiple_event % 2 == 0:
        temp = filtered_df_cust.groupby([filtered_df_cust[START_DATE_COL].dt.strftime('%Y-%m'), filtered_df_cust[CUST_ID_COL]]).size().reset_index(name='count')
        temp[START_DATE_COL] = pd.to_datetime(temp[START_DATE_COL])
        temp = temp.groupby([temp[START_DATE_COL].dt.strftime('%Y-%m'), temp['count']]).size().reset_index(name='cust_count')
        temp[START_DATE_COL] = pd.to_datetime(temp[START_DATE_COL])

        multiple_event_client_fig = go.Figure()
        for group in sorted(temp['count'].unique()):
            x = temp[temp['count']==group][START_DATE_COL]
            y = temp[temp['count']==group]['cust_count']
            name = str(int(group)) + 'Agreement'
            multiple_event_client_fig.add_trace(go.Bar(x=x, y=y, name=name))

        multiple_event_client_fig.update_layout(
        title='How many customers have multiple agreements in Each Month',
        yaxis_title='Number of Customers',
        title_x=0.5,
        barmode='stack')
        multiple_event_client_fig.update_xaxes(dtick='M1', tickformat="%b\n%Y")

    else:
        temp = filtered_df_cust.groupby([filtered_df_cust[START_DATE_COL].dt.year, filtered_df_cust[CUST_ID_COL]]).size().reset_index(name='count')
        temp = temp.groupby([START_DATE_COL, 'count']).size().reset_index(name='cust_count')

        multiple_event_client_fig = go.Figure()
        for group in temp['count'].unique():
            x = temp[temp['count']==group][START_DATE_COL]
            y = temp[temp['count']==group]['cust_count']
            name = str(int(group)) + 'Agreement'
            multiple_event_client_fig.add_trace(go.Bar(x=x, y=y, name=name))

        multiple_event_client_fig.update_layout(
        title='How many customers have multiple agreements in Each Year',
        yaxis_title='Number of Customers',
        title_x=0.5,
        barmode='stack')
        multiple_event_client_fig.update_xaxes(dtick='Y1', tickformat="%b\n%Y")

    return [multiple_event_client_fig]



# Layout for Tab3: Event Value
app2 = dash.Dash(__name__)
app2.layout = html.Div(
    [
        html.H1('Event Value vs Time', style={'textAlign': 'center'}),
        html.Div(
                [dmc.DateRangePicker(
                    id="date-picker-range",
                    label="Date Range",
                    minDate=pd.Timestamp(df[START_DATE_COL].min()),
                    maxDate=pd.Timestamp(df[START_DATE_COL].max()),
                    value=[pd.Timestamp(df[START_DATE_COL].min()), pd.Timestamp(df[START_DATE_COL].max())],
                    style={"width": 350},
                ),
                dmc.Space(h=10)]
        ),
        dcc.Dropdown(id='product-dropdown', options=[{'label': str(product), 'value': product} for product in sorted(df[PRODUCT_TYPE].unique())],
                    placeholder='Select Product(s)'),
        html.P("Event Value Threshold Rangeslider", style={'textAlign': 'center'}),
        dcc.RangeSlider(
            min=df[OPENING_AMOUNT_COL].min(),
            max=df[OPENING_AMOUNT_COL].max(),
            step=(df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10,
            id='threshold-slider',
            value=[df[OPENING_AMOUNT_COL].min(), df[OPENING_AMOUNT_COL].max()],
            marks={i: str(i) for i in range(int(df[OPENING_AMOUNT_COL].min()), int(df[OPENING_AMOUNT_COL].max()), round((df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10))}
        ),
        html.Button('Month/Year/Week/Day', id='day-button', n_clicks=0),
        html.P('What is the average event value per day/month/year/within specific time?'),
        dcc.Graph(id='average-event-value-chart'),
        html.P('What is the average event value for each day of the week?'),
        html.P('How many customers open on each day of the week?'),
        html.Button('Average Event Value/Event Count', id='avg-count-button1', n_clicks=0),
        dcc.Graph(id='average-day-week-chart'),
        html.P('What is the average event value for each day of the month?'),
        html.P('How many customers open on each day of the month?'),
        html.Button('Average Event Value/Event Count', id='avg-count-button2', n_clicks=0),
        dcc.Graph(id='average-day-month-chart'),
        html.P('What is the average event value for each month of the year?'),
        html.P('How many customers open on each month of the year?'),
        html.Button('Average Event Value/Event Count', id='avg-count-button3', n_clicks=0),
        dcc.Graph(id='average-month-year-chart'),
        html.P('What is the average event value for each quarter of the year?'),
        html.P('How many customers open on each quarter of the year?'),
        html.Button('Average Event Value/Event Count', id='avg-count-button4', n_clicks=0),
        dcc.Graph(id='average-quarter-month-chart'),
        html.P('What is the average event_value for each client (by year)?'),
        dcc.Graph(id='avg-value-client-chart'),
        html.P('What is the distribution of event_values?'),
        html.P('How does the event_value distribution vary within different periods?'),
        dcc.Graph(id='distribution-chart'),
        html.P('What is the total event_value within a specific time (e.g., last 30 days)?'),
        dcc.Graph(id='total-value-chart')
    ]
)

@app.callback([Output('average-event-value-chart', 'figure')],
              [Input('date-picker-range', 'value'),
              Input('product-dropdown', 'value'),
              Input('threshold-slider', 'value'),
              Input('day-button', 'n_clicks')]
)

def box_chart(date_range, selected_product, threshold_range, day_clicks):
    # Filter the dataframe based on the slider range
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Determine the grouping based on button clicks
    if day_clicks % 4 == 0:
        grouping = filtered_df[START_DATE_COL].dt.strftime('%Y-%m')
        x_title = 'Month'
        dtick_format = "M1"
        tick_format = "%b\n%Y"

    elif day_clicks % 4 == 1:
        grouping = filtered_df[START_DATE_COL].dt.year
        x_title = 'Year'
        dtick_format = "Y1"
        tick_format = "%b\n%Y"

    elif day_clicks % 4 == 2:
        week_df = filtered_df.sort_values(START_DATE_COL)
        grouping = week_df[START_DATE_COL].dt.strftime('%Y-%W')
        x_title = 'Week'
        dtick_format = None
        tick_format = None

    else:
        temp = filtered_df.groupby(START_DATE_COL)[OPENING_AMOUNT_COL].mean()
        x_title = 'Day'
        dtick_format = None
        tick_format = None

    # Create the box chart
    if day_clicks % 4 == 2:
        box_chart = go.Figure(data=go.Box(x=grouping, y=filtered_df[OPENING_AMOUNT_COL], boxmean=True, name='Bar Plot'))
        box_chart.update_layout(
            xaxis_title=x_title,
            yaxis_title='Event Value',
            title='Average Event Value vs Time',
            title_x=0.5,
            xaxis={'type': 'category'}
        )
        box_chart.update_xaxes(dtick=dtick_format, tickformat=tick_format)
        box_chart.add_trace(go.Scatter(x=filtered_df.groupby(grouping)[OPENING_AMOUNT_COL].mean().index,
                                       y=filtered_df.groupby(grouping)[OPENING_AMOUNT_COL].mean().values,
                                       name='Mean'))
    elif day_clicks % 4 == 0 or day_clicks % 4 == 1:
        box_chart = go.Figure(data=go.Box(x=grouping, y=filtered_df[OPENING_AMOUNT_COL], boxmean=True, name='Bar Plot'))
        box_chart.update_layout(
            xaxis_title=x_title,
            yaxis_title='Event Value',
            title='Average Event Value vs Time',
            title_x=0.5
        )
        box_chart.update_xaxes(dtick=dtick_format, tickformat=tick_format)
        box_chart.add_trace(go.Scatter(x=filtered_df.groupby(grouping)[OPENING_AMOUNT_COL].mean().index,
                                       y=filtered_df.groupby(grouping)[OPENING_AMOUNT_COL].mean().values,
                                       name='Mean'))
    else:
        box_chart = go.Figure(data=go.Scatter(x=temp.index, y=temp.values, mode='lines'))
        box_chart.update_layout(
        xaxis_title=x_title,
        yaxis_title='Event Value',
        title='Average Event Value vs Time',
        title_x=0.5
        )
        box_chart.update_xaxes(dtick=dtick_format, tickformat=tick_format)

    return [box_chart]

@app.callback(
    [Output('average-day-week-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value'),
     Input('avg-count-button1', 'n_clicks')]
)

def fig_day_week(date_range, selected_product, threshold_range, avg_count_button1):
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Day_Week
    day_sequence = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']

    if avg_count_button1 % 2 == 0:
        fig_day_week = go.Figure(data=go.Box(x=filtered_df[START_DATE_COL].dt.day_name(), y=filtered_df[OPENING_AMOUNT_COL], boxmean=True))
        fig_day_week.update_layout(
            title='Average Event Value by Day of Week',
            xaxis_title='Day of Week',
            yaxis_title='Event Value',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': day_sequence}
        )
    else:
        events_count_day_week = filtered_df[START_DATE_COL].dt.day_name().value_counts().reindex(day_sequence, fill_value=0)
        fig_day_week = go.Figure(data=go.Bar(x=events_count_day_week.index, y= events_count_day_week.values))
        fig_day_week.update_layout(
            title='Event Count by Day of Week',
            xaxis_title='Day of Week',
            yaxis_title='Event Count',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': day_sequence}
        )

    return [fig_day_week]

@app.callback(
    [Output('average-day-month-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value'),
     Input('avg-count-button2', 'n_clicks')]
)

def fig_day_month(date_range, selected_product, threshold_range, avg_count_button2):
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Day_Month
    if avg_count_button2 % 2 == 0:
        fig_day_month = go.Figure(data=go.Box(x=filtered_df[START_DATE_COL].dt.day, y=filtered_df[OPENING_AMOUNT_COL], boxmean=True))
        fig_day_month.update_layout(
            title='Average Event Value by Day of Month',
            xaxis_title='Day of Month',
            yaxis_title='Event Value',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': [str(i) for i in range(1, 32)]}
        )
    else:
        events_count_day_month = filtered_df[START_DATE_COL].dt.day.value_counts().reindex(fill_value=0)
        fig_day_month = go.Figure(data=go.Bar(x=events_count_day_month.index, y= events_count_day_month.values))
        fig_day_month.update_layout(
            title='Event Count by Day of Month',
            xaxis_title='Day of Month',
            yaxis_title='Event Count',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': [str(i) for i in range(1, 32)]}
        )

    return [fig_day_month]

@app.callback(
    [Output('average-month-year-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value'),
     Input('avg-count-button3', 'n_clicks')]
)

def fig_month_year(date_range, selected_product, threshold_range, avg_count_button3):
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Month_Year
    month_sequence = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    if avg_count_button3 %2 == 0:
        fig_month_year = go.Figure(data=go.Box(x=filtered_df[START_DATE_COL].dt.month_name(), y=filtered_df[OPENING_AMOUNT_COL], boxmean=True))
        fig_month_year.update_layout(
            title='Average Event Value by Month of Year',
            xaxis_title='Month',
            yaxis_title='Event Value',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': month_sequence}
        )
    else:
        events_count_month_year = filtered_df[START_DATE_COL].dt.month_name().value_counts().reindex(month_sequence, fill_value=0)
        fig_month_year = go.Figure(data=go.Bar(x=events_count_month_year.index, y= events_count_month_year.values))
        fig_month_year.update_layout(
            title='Event Count by Month of Year',
            xaxis_title='Month',
            yaxis_title='Event Count',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': month_sequence}
        )

    return [fig_month_year]

@app.callback(
    [Output('average-quarter-month-chart','figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value'),
     Input('avg-count-button4', 'n_clicks')]
)

def fig_quarter_year(date_range, selected_product, threshold_range, avg_count_button4):
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Quarter_Year

    if avg_count_button4 % 2 == 0:
        fig_quarter_year = go.Figure(data=go.Box(x=filtered_df[START_DATE_COL].dt.quarter, y=filtered_df[OPENING_AMOUNT_COL], boxmean=True))
        fig_quarter_year.update_layout(
            title='Average Event Value by Quarter of Year',
            xaxis_title='Quarter',
            yaxis_title='Event Value',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': [1, 2, 3, 4]}
        )
    else:
        events_count_quarter_year = filtered_df[START_DATE_COL].dt.quarter.value_counts().reindex(fill_value=0)
        fig_quarter_year = go.Figure(data=go.Bar(x=events_count_quarter_year.index, y= events_count_quarter_year.values))
        fig_quarter_year.update_layout(
            title='Event Count by Quarter of Year',
            xaxis_title='Quarter',
            yaxis_title='Event Count',
            title_x=0.5,
            xaxis={'categoryorder': 'array', 'categoryarray': [1, 2, 3, 4]}
        )

    return [fig_quarter_year]

@app.callback(
    [Output('avg-value-client-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value')]
)

def avg_value_client_fig(date_range, selected_product, threshold_range):
    starting_date, ending_date = date_range
    # Filter the dataframe based on the slider range and selected dates
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Average event value for each client
    temp = filtered_df.groupby(CUST_ID_COL)[OPENING_AMOUNT_COL].mean()
    temp_df = pd.DataFrame(temp)
    temp_df['Bin'] = pd.qcut(temp_df[OPENING_AMOUNT_COL], q=20, duplicates='drop')
    bin_count = temp_df['Bin'].value_counts().sort_index()
    x = bin_count.index
    y = bin_count.values

    # Convert Interval objects to string representation
    x_ticktext = [str(interval) for interval in x]

    avg_value_client_fig = go.Figure(data=[go.Bar(x=x_ticktext, y=y)])
    avg_value_client_fig.update_layout(
        title='Average Event Value for Each Client by Quantile',
        xaxis_title='Average Event Value (Bin)',
        yaxis_title='Number of Counts',
        title_x=0.5
    )
    return [avg_value_client_fig]

@app.callback(
    [Output('distribution-chart', 'figure'),
     Output('total-value-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value')]
)

def update_chart(date_range, selected_product, threshold_range):
    starting_date, ending_date = date_range
    filtered_df = df[
                    (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
                    (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
                    (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
                    (df[START_DATE_COL] <= pd.to_datetime(ending_date))
                    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    # Distribution of Event Value
    """ quantiles = [0, 0.1, 0.2, 0.5, 0.8, 1]
    quantile_values = []

    for quantile in quantiles:
        quantile_value = df[OPENING_AMOUNT_COL].quantile(quantile)
        quantile_values.append(quantile_value) """

    cust_filtered_df = filtered_df.sort_values(by=[CUST_ID_COL, START_DATE_COL]).copy()
    cust_filtered_df['rank'] = cust_filtered_df.groupby(CUST_ID_COL)[START_DATE_COL].rank(ascending=True, method='first')
    cust_filtered_df['Bin'] = pd.qcut(cust_filtered_df[OPENING_AMOUNT_COL], q=20, duplicates='drop')

    fig_distribution = go.Figure()
    for group in sorted(cust_filtered_df['rank'].unique()):
        bin_count = cust_filtered_df[cust_filtered_df['rank']==group]['Bin'].value_counts().sort_index()
        x = bin_count.index
        x_ticktext = [str(interval) for interval in x]
        y = bin_count.values
        name = str(int(group)) + 'Agreement'
        fig_distribution.add_trace(go.Bar(x=x_ticktext, y=y, name=name))

    fig_distribution.update_layout(
        title='Event Value Distribution by Quantile',
        xaxis_title='Event Value (Bin)',
        yaxis_title='Event Count',
        title_x=0.5,
        barmode='stack')

    # Total Event Value within Specific Time
    filtered_df_total_event = filtered_df.sort_values(START_DATE_COL)
    rolling_cumsum = filtered_df_total_event.groupby(START_DATE_COL)[OPENING_AMOUNT_COL].sum().expanding().sum()

    fig_total_event_value = go.Figure(data=go.Bar(x=filtered_df.sort_values(START_DATE_COL)[START_DATE_COL].unique(), y=rolling_cumsum))
    fig_total_event_value.update_layout(
        title='Cumulative Event Value',
        xaxis_title='Date',
        yaxis_title='Cumulative Event Value',
        title_x=0.5
    )

    return (fig_distribution, fig_total_event_value)


# Layout for Tab4: Time Difference
app3 = dash.Dash(__name__)
app3.layout = html.Div(
    [
        html.H1('Event Time Difference', style={'textAlign': 'center'}),
        html.Div(
                [dmc.DateRangePicker(
                    id="date-picker-range",
                    label="Date Range",
                    minDate=pd.Timestamp(df[START_DATE_COL].min()),
                    maxDate=pd.Timestamp(df[START_DATE_COL].max()),
                    value=[pd.Timestamp(df[START_DATE_COL].min()), pd.Timestamp(df[START_DATE_COL].max())],
                    style={"width": 350},
                ),
                dmc.Space(h=10)]
        ),
        dcc.Dropdown(id='product-dropdown', options=[{'label': str(product), 'value': product} for product in sorted(df[PRODUCT_TYPE].unique())],
                     placeholder='Select Product(s)'),
        html.P("Event Value Threshold Rangeslider", style={'textAlign': 'center'}),
        dcc.RangeSlider(
            min=df[OPENING_AMOUNT_COL].min(),
            max=df[OPENING_AMOUNT_COL].max(),
            step=(df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10,
            id='threshold-slider',
            value=[df[OPENING_AMOUNT_COL].min(), df[OPENING_AMOUNT_COL].max()],
            marks={i: str(i) for i in range(int(df[OPENING_AMOUNT_COL].min()), int(df[OPENING_AMOUNT_COL].max()), round((df[OPENING_AMOUNT_COL].max()-df[OPENING_AMOUNT_COL].min())/10))}
        ),
        html.P("What is the average time between events for each client?"),
        dcc.Graph(id='average-time-client-chart'),
        html.Button('Max/Min', id='max-min-button', n_clicks=0),
        dcc.Graph(id='maxmin-time-client-chart')
    ]
)

@app.callback(
    [Output('average-time-client-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value')]
)

def client_avg_time_fig(date_range, selected_product, threshold_range):

    # Filter the dataframe based on the slider range and selected dates
    starting_date, ending_date = date_range
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    filtered_df_cust = filtered_df.copy().sort_values([CUST_ID_COL, START_DATE_COL], ascending=[True, True])

    # Average time diff between events for each client
    filtered_df_cust['time_diff'] = filtered_df_cust.groupby(CUST_ID_COL)[START_DATE_COL].diff()
    filtered_df_cust['time_diff'] = filtered_df_cust['time_diff'] /np.timedelta64(1,"D")
    nan_count = filtered_df_cust.drop_duplicates(subset=CUST_ID_COL, keep=False)['time_diff'].isna().sum() #  count only customers with one CD

    histogram_trace = go.Histogram(x=filtered_df_cust['time_diff'], name='Time Difference')
    nan_count_trace = go.Bar(x=['-1'], y=[nan_count], name='NaN Count', visible='legendonly')
    client_avg_time_fig = go.Figure(data=[histogram_trace, nan_count_trace])

    client_avg_time_fig.update_layout(
        title='Average Time Difference Between Events for Each Client',
        xaxis_title='Day Difference',
        yaxis_title='Number of Counts',
        title_x=0.5
    )

    return [client_avg_time_fig]

@app.callback(
    [Output('maxmin-time-client-chart', 'figure')],
    [Input('date-picker-range', 'value'),
     Input('product-dropdown', 'value'),
     Input('threshold-slider', 'value'),
     Input('max-min-button', 'n_clicks')]
)

def client_maxmin_time_fig(dates, selected_product, threshold_range, max_min_button):

    # Filter the dataframe based on the slider range and selected dates
    starting_date, ending_date = dates
    filtered_df = df[
        (df[OPENING_AMOUNT_COL] >= threshold_range[0]) &
        (df[OPENING_AMOUNT_COL] <= threshold_range[1]) &
        (df[START_DATE_COL] >= pd.to_datetime(starting_date)) &
        (df[START_DATE_COL] <= pd.to_datetime(ending_date))
    ].copy()

    if selected_product:
        filtered_df = filtered_df[filtered_df[PRODUCT_TYPE]==selected_product]
    else:
        filtered_df = filtered_df

    filtered_df_cust = filtered_df.copy().sort_values([CUST_ID_COL, START_DATE_COL], ascending=[True, True])

    filtered_df_cust['time_diff'] = filtered_df_cust.groupby(CUST_ID_COL)[START_DATE_COL].diff()
    filtered_df_cust['time_diff'] = filtered_df_cust['time_diff'] /np.timedelta64(1,"D")

    nan_count = filtered_df_cust.drop_duplicates(subset=CUST_ID_COL, keep=False)['time_diff'].isna().sum() #  count only customers with one CD
    histogram_trace = go.Histogram(x=filtered_df_cust['time_diff'], name='Time Difference')
    nan_count_trace = go.Bar(x=['-1'], y=[nan_count], name='NaN Count', visible='legendonly')

    # Max and Min time diff between events for each client
    if max_min_button % 2 == 0:
        histogram_trace = go.Histogram(x=filtered_df_cust.groupby(CUST_ID_COL)['time_diff'].max(), name='Max Time Difference')
        nan_count_trace = go.Bar(x=['-1'], y=[nan_count], name='NaN Count', visible='legendonly')
        client_maxmin_time_fig = go.Figure(data=[histogram_trace, nan_count_trace])
        client_maxmin_time_fig.update_layout(
        title='Max Time Difference Between Events for Each Client',
        xaxis_title='Day Difference',
        yaxis_title='Number of Counts',
        title_x=0.5
    )
    else:
        histogram_trace = go.Histogram(x=filtered_df_cust.groupby(CUST_ID_COL)['time_diff'].min(), name='Min Time Difference')
        nan_count_trace = go.Bar(x=['-1'], y=[nan_count], name='NaN Count', visible='legendonly')
        client_maxmin_time_fig = go.Figure(data=[histogram_trace, nan_count_trace])
        client_maxmin_time_fig.update_layout(
        title='Min Time Difference Between Events for Each Client',
        xaxis_title='Day Difference',
        yaxis_title='Number of Counts',
        title_x=0.5
    )

    return [client_maxmin_time_fig]


# Define the layout for the main app
app.layout = html.Div([
    dcc.Tabs(id='tabs', value='app0', children=[
        dcc.Tab(label='General Statistics', value='app0'),
        dcc.Tab(label='Event Count vs Time', value='app1'),
        dcc.Tab(label='Event Value vs Time', value='app2'),
        dcc.Tab(label='Event Time Difference', value='app3')

    ]),
    html.Div(id='tab-content')
])

# Define the callback to switch between tabs
@app.callback(Output('tab-content', 'children'), [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'app0':
        return app0.layout
    elif tab == 'app1':
        return app1.layout
    elif tab == 'app2':
        return app2.layout
    elif tab == 'app3':
        return app3.layout

# Run the main app
if __name__ == '__main__':
    app.run_server(debug=False)



<IPython.core.display.Javascript object>